<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Fasttext0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install unidecode
!pip install word2number
!pip install contractions

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=beff3aea39d80836f467cac9c7d0b1a0d2525996dd51696290e490707b2583f5
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 284 kB 8.1 MB/s 
     |████████████████████████████████| 321 kB 66.8 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=ac55d29d01d703d087d88315a9bf7a8bafba6aa7f47f095315e1fe5af53cd6e6
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [6]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1
df= df_fake.head(50).append(df_real.head(50))
df.reset_index(drop=True, inplace=True)

In [53]:
print(df.columns)

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML'], dtype='object')


In [46]:
#print(df)

                      id  ... labelML
0   gossipcop-2493749932  ...       0
1   gossipcop-4580247171  ...       0
2    gossipcop-941805037  ...       0
3   gossipcop-2547891536  ...       0
4   gossipcop-5476631226  ...       0
..                   ...  ...     ...
95      gossipcop-955009  ...       1
96      gossipcop-897603  ...       1
97      gossipcop-908402  ...       1
98      gossipcop-862967  ...       1
99      gossipcop-886127  ...       1

[100 rows x 5 columns]


In [54]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)

In [55]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [50]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
#---------df1.iloc[:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__' + x)
print(df1)

                                                title  labelML
0                                                 NaN        0
1   __label__paris jackson cara delevingne enjoy n...        0
2   __label__celebrities join tax march in protest...        0
3   __label__cindy crawford daughter kaia gerber w...        0
4     __label__full list of oscar nominations variety        0
..                                                ...      ...
95  __label__michael buble and wife luisana lopila...        1
96  __label__selena gomez is going to keep her blo...        1
97   __label__netflix new releases coming in february        1
98  __label__halsey calls iggy azalea king moron a...        1
99  __label__delilah opens up about son suicide an...        1

[100 rows x 2 columns]



 #df = df.loc[:,"title"]
 # just for loc practice, ignore.


In [56]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:49, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__0 ' + x)
df1.iloc[50:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__1 ' + x)
print(df1)

                                                title  labelML
0                                                 NaN        0
1   __label__0 paris jackson cara delevingne enjoy...        0
2   __label__0 celebrities join tax march in prote...        0
3   __label__0 cindy crawford daughter kaia gerber...        0
4   __label__0 full list of oscar nominations variety        0
..                                                ...      ...
95  __label__1 michael buble and wife luisana lopi...        1
96  __label__1 selena gomez is going to keep her b...        1
97  __label__1 netflix new releases coming in febr...        1
98  __label__1 halsey calls iggy azalea king moron...        1
99  __label__1 delilah opens up about son suicide ...        1

[100 rows x 2 columns]


In [58]:
df1.reset_index(drop=True, inplace=True)
X, y = df1.iloc[:, 1:], df1['labelML']

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 1.000000
